# [Can You Survive Another Squid Game?](https://thefiddler.substack.com/p/can-you-survive-another-squid-game)
## July 21 2023

# Problem

_This week’s puzzle is an oldie but a goodie, and comes from high school student Meryl Zhang of Plano, Texas, who was named the top math awardee at this year’s Regeneron International Science and Engineering Fair. For her project, Meryl developed novel heuristics for the NP-hard partition problem. Meryl enjoys K-dramas, including Squid Game—a show that has already proven ripe for riddles._

_This week, the final 20 players in the Squid Game competition form a circle and are assigned the whole numbers from 1 to 20, progressing in a clockwise direction. First, contestant 2 is eliminated. Then, the contestant two positions clockwise from where 2 was (i.e., contestant 4) is eliminated. Next, the contestant two positions clockwise from them (i.e., contestant 6) is eliminated. The counting continues in this manner, wrapping around the circle, which tightens after each elimination. So after contestant 20 is eliminated, the next contestant to go is 3, who at this point is two positions clockwise from where 20 once stood._

_You repeat this process until only one contestant remains as the ultimate winner of the game. What is the winner’s number?_

## Solution

After tinkering with some more mathematical approaches and not getting anywhere, I decided to see if any patterns would emerge for various numbers of players so I wrote up some code real quick to run the game. Interestingly, the following pattern emerged. 

| No. Players | Winner |
| --- | --- |
| 2 | 1
| 3 | 3
| 4 | 1
| 5 | 3
| 6 | 5
| 7 | 7
| 8 | 1
| 9 | 3
| 10 | 5
| 11 | 7
| 12 | 9
| 13 | 11
| 14 | 13
| 15 | 15
| 16 | 1

In [11]:
class Node(object):
    def __init__(self, n:int, next=None, prev=None):
        self.n = n
        self.next = next
        self.prev = prev

class Circle(object):
    def __init__(self, n:int):
        self.num_nodes = n
        self.head = Node(1)
        self.current_spot = self.head

        prev = self.head

        for i in range(2,n+1):
            prev.next = Node(i)
            prev.next.prev = prev
            prev = prev.next
        
        prev.next = self.head
        self.head.prev = prev
        return None
    
    def _remove_current(self):
        self.num_nodes -= 1
        # print(self.current_spot.n)
        self.current_spot.prev.next, self.current_spot.next.prev = (self.current_spot.next, self.current_spot.prev)
        self.current_spot = self.current_spot.next
    
    def _advance(self):
        if self.num_nodes > 1:
            self.current_spot = self.current_spot.next
            self._remove_current()
        else:
            raise Exception("There is only one node left")
    
    def run_game(self):
        while self.num_nodes > 1:
            self._advance()
        return self.current_spot.n


for i in range(2,70+1):
    c = Circle(i)
    print(c.run_game())

1
3
1
3
5
7
1
3
5
7
9
11
13
15
1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
63
1
3
5
7
9
11
13


Would you look at that! Definitely a pattern! A few things pop out:

  -1. Each subsequent answer increases by $2$ which also happens to be the step size
  -2. Obviously the answer never exceeds the number of players %n%

This suggests a recursive solution relating $f(n, s) = g( f(n-1, s)$. We just need to figure out the function $g$.

Using these observations, I'm inclined to suggest $f(n,s) = (f(n-1,s) + s) \mod n$

However, when we run this we find that it doesn't quite give the correct answer. It appears that our indexing is a little off.  We get an answer of $0$ for $n=2$ which makes sense upon closer inspection of our equation.  So let's adjust our equation a little to account for the fact that the problem is 1-indexed (whereas modulo more naturally supports 0-indexing).

$ f(n,s) = \big( (f(n-1,s) + (s-1) ) \mod n \big) + 1 $

In [13]:
def calc(n,s):
    if n==1:
        return 1
    
    return (calc(n-1, s) + s-1) % n + 1

for i in range(2,20+1):
    print(calc(i,2))

1
3
1
3
5
7
1
3
5
7
9
11
13
15
1
3
5
7
9


Perfect! If we check other step sizes, it also works! But lets take a momement to go back and try to better understand why this works (we did sort of stumble on the solution based on the results of various manual calculations). 

If we imagine a game with $n$ players for step $s$ (i.e. $f(n,s)=a$ where $a$ is the final remaining location), then after we've eliminated the first player, we have advanced $k$ spots and now have $n-1$ players. So the state of the board resembles the original state $f(n,s)$ if we started the $f(n,s)$ game $k$ steps forward (i.e. $f(n,s)+k$) and had one fewer player $(f(n-1,s)$). Together we intuitively have $f(n,s) = (f(n-1,s) + k) \mod n$. Barring the indexing issue,  this intuition appears to bring us to the correct answer.

# Extra Credit


My first thought was that we could calculating the stationary distribution of a markov chain with a relationship like $P(n) = \frac{1}{2}P(n-2) + \frac{1}{2}P(n-3)$. Of course, this formulation doesn't actually work because after the first round, $P(n)$'s predecessors aren't confined to $P(n-2)$ and $P(n-3)$. In fact, at the start, it's not clear that we should exclude the possiblity of _any_ player transitioning to any other player.

Another approach would be to brute force calculation.  However this undergoes exponential growth (e.g. $O(2^n)$).

Unfortunately, I'm running out of time to find the answer using an analytic approach so I'm just going to simulate the problem like a weenie. 

In [14]:
import random

class RandomCircle(Circle):
    def __init__(self, n:int):
        super().__init__(n)

    def _advance(self):
        s = random.choice([2,3])
        if self.num_nodes > 1:
            for _ in range(s-1):
                self.current_spot = self.current_spot.next
            self._remove_current()
        else:
            raise Exception("There is only one node left")

In [20]:
from collections import defaultdict

results = defaultdict(lambda: 0)
n = 1000000
for _ in range(n):
    r = RandomCircle(20)
    winner = r.run_game()
    results[winner] += 1

for k, v in sorted(results.items(), key=lambda a: a[1], reverse=True):
    print(f"Player {k} wins {v/n}% of the time.")

Player 18 wins 0.068046% of the time.
Player 19 wins 0.066954% of the time.
Player 17 wins 0.066689% of the time.
Player 20 wins 0.065004% of the time.
Player 16 wins 0.063649% of the time.
Player 1 wins 0.062299% of the time.
Player 15 wins 0.060377% of the time.
Player 14 wins 0.05672% of the time.
Player 13 wins 0.052413% of the time.
Player 12 wins 0.050493% of the time.
Player 11 wins 0.046386% of the time.
Player 4 wins 0.04563% of the time.
Player 10 wins 0.043576% of the time.
Player 9 wins 0.043298% of the time.
Player 7 wins 0.039847% of the time.
Player 6 wins 0.03928% of the time.
Player 8 wins 0.037527% of the time.
Player 5 wins 0.030916% of the time.
Player 2 wins 0.030693% of the time.
Player 3 wins 0.030203% of the time.


Assuming my code was correct, Player 18 has the best chance of winning at around $6.8%$. 